# Identifying and Extracting Longitudinal Variables using R PIC-SURE API

This tutorial notebook will demonstrate how to idetify and extract longitudinal variables using the R PIC-SURE API. Longitudinal variables are defined as containing multiple 'Exam' or 'Visit' descriptions within their concept path. 

In this example, we will find the patient level data for a lipid-related longitudinal variable within the Framingham Heart study. We will:
1. Identify what longitudinal variables are associated with the keywords of interest (lipid, triglyceride), and how many exams / visits are associated with each one
2. Select a longitudinal variable of interest from a specific study (Framingham heart study)
3. Extract patient level data into a dataframe where each rows represent patients and columns represent visits

For a more basic introduction to the R PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to get a user-specific security token. For more information about how to proceed, see the "Get your security token" instructions in the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/harmonized_lipid_measurements_example/NHLBI_BioData_Catalyst#get-your-security-token).**

## Environment Set-Up

### System Requirements
R >= 3.4

### Install Packages

In [ ]:
source("R_lib/requirements.R")

Install latest R PIC-SURE API libraries from github

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
install.packages("https://cran.r-project.org/src/contrib/Archive/devtools/devtools_1.13.6.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/R6_2.5.0.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/hash_2.2.6.1.tar.gz", repos=NULL, type="source")
install.packages(c("urltools"),repos = "http://cran.us.r-project.org")
devtools::install_github("hms-dbmi/pic-sure-r-client", force=T)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force=T)
devtools::install_github("hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds", force=T)

Load user-defined functions

In [ ]:
source("R_lib/utils.R")

## Connecting to a PIC-SURE Network
**Again, before running this notebook, please be sure to get a user-specific security token. For more information about how to proceed, see the "Get your security token" instructions in the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/harmonized_lipid_measurements_example/NHLBI_BioData_Catalyst#get-your-security-token).**

In [ ]:
PICSURE_network_URL <- "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id <- "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file <- "token.txt"

In [ ]:
token <- scan(token_file, what = "character")


In [ ]:
myconnection <- picsure::connect(url = PICSURE_network_URL,
                                 token = token)

In [ ]:
resource <- bdc::get.resource(myconnection,
                               resourceUUID = resource_id)

## Longitudinal Lipid Variable Example
Example showing how to extract lipid measurements from multiple visits for different cohorts

### Access the data
First, we will create a multiIndex variable dictionary of all variables we have access to.

In [ ]:
varDict <- bdc::find.in.dictionary(resource) %>% bdc::extract.entries() # all variables
multiindex <- get_multiIndex_variablesDict(varDict) # get multiindex table of all variables

In this example, we are interested in variables related to lipids. We can find all variables related to the search terms 'lipid' and 'triglyceride' through applying the following filter on the multiIndex dictionary:

In [ ]:
lipid_vars <- multiindex %>% filter(grepl('triglyceride', name, ignore.case = TRUE) |
                                   grepl('lipid', name, ignore.case = TRUE))
lipid_vars

### Identify the longitudinal lipid variables
This block of code does the following:
- uses the multiindex dataframe containing variables which are related to 'lipid' or 'triglyceride'
- filters for variables with keywords 'exam #' or 'visit #'
- extracts the exam number of each variable into column `exam_number`
- groups variables by study (`level_0`) and longitudinal variable (`longvar`)
- returns a table showing the variables that have more than one exam recorded (longitudinal variables?)

In [ ]:
lipid_vars %>% 
  filter((grepl('Exam \\d+', name, ignore.case = TRUE) | 
         grepl('Visit \\d+', name, ignore.case = TRUE))) %>%
  mutate(exam_number = str_extract(name, regex("(exam \\d)|(visit \\d)", ignore_case=T)),
         # create new var 'longvar' without exam / visit number
         longvar =  str_replace(name, regex('(exam \\d)|(visit \\d)', ignore_case = T), '')) %>%
  group_by(level_0, longvar) %>%
  summarise(n_exams = n_distinct(exam_number)) %>% 
  # filter results to include n_exams greater than 1 to find longitudinal vars
  filter(n_exams > 1)
   

We can filter for specific studies after this and then extract the longitudinal variable names (note that longvar is not equivalent to the actual PIC-SURE concept path, will need to use original name from lipid vars); you won't be able to use the table above by itself to get the data of interest

### Add lipid variables to query
